# Create spotify playlist and add tracks by ID

Spotipy documentation: https://spotipy.readthedocs.io/en/latest/

Spotify API documentation: https://developer.spotify.com/web-api/

In [1]:
import spotipy, spotipy.util as util, pandas as pd, math, time
from keys import spotify_username, spotify_client_id, spotify_client_secret, spotify_redirect_uri

In [2]:
pause = 0.5 #how long to pause between requests
limit = 100 #number of tracks to add per request (max=100)
max_playlist_size = 11000 #max number of tracks spotify allows in a single playlist

## Load the playlists and de-dupe by artist id

In [3]:
df = pd.read_csv('data/playlist_tracks.csv', encoding='utf-8')
print(len(df))
df['playlist_id'].value_counts()

19224


2UfBZ8aRbMniK0052b5uYb    9872
25suCiMhFOWCWLG2JLfhot    9352
Name: playlist_id, dtype: int64

In [4]:
df = df.sort_values(by='duration_ms', ascending=True)

In [5]:
df_unique = df.drop_duplicates(subset='artist_id', keep='first')
df_unique = df_unique.sort_values(by='duration_ms', ascending=False)
print(len(df_unique))
df_unique['playlist_id'].value_counts()

11304


2UfBZ8aRbMniK0052b5uYb    5942
25suCiMhFOWCWLG2JLfhot    5362
Name: playlist_id, dtype: int64

## Authorize spotipy and create connection

In [6]:
# define scope to work with playlists then get an auth token
scope = 'playlist-read-private playlist-modify-private playlist-modify-public'
token = util.prompt_for_user_token(spotify_username, scope, spotify_client_id, spotify_client_secret, spotify_redirect_uri)

In [7]:
# open a connection to spotify
sp = spotipy.Spotify(auth=token)

## Create playlist track ID lists (max 10,000 per playlist)

In [8]:
# get the user id associated with this username
user = sp.user(spotify_username)
user_id = user['id']

In [9]:
all_tracks = list(df_unique['track_id'].values)
len(all_tracks)

11304

In [10]:
# if length > max_playlist_size, you need multiple playlists
playlists_tracks = []
for n in range(math.ceil(len(all_tracks) / max_playlist_size)):
    offset = n * max_playlist_size
    playlists_tracks.append(all_tracks[offset:offset+max_playlist_size])

## Create new playlists and add all the tracks to it

In [11]:
current_time = int(time.time())
for playlist_tracks, i in zip(playlists_tracks, range(len(playlists_tracks))):
    
    new_playlist_name = 'Big List {}_{}'.format(current_time, i+1)
    result = sp.user_playlist_create(user=user_id, name=new_playlist_name, public=False)
    playlist_id = result['id']
    print('Created playlist "{}", ID={}'.format(new_playlist_name, playlist_id))
          
    call_count = math.ceil(len(playlist_tracks) / limit)
    tracks_added = []
    for n in range(call_count):
        time.sleep(pause)
        offset = n * limit
        tracks_batch = playlist_tracks[offset:offset+limit]
        snapshot_id = sp.user_playlist_add_tracks(user=user_id, playlist_id=playlist_id, tracks=tracks_batch)
        tracks_added.extend(tracks_batch)
        
    print('  Added {:,} tracks to "{}"'.format(len(tracks_added), new_playlist_name))

Created playlist "Big List 1466376055_1", ID=4cAJGG9Ipf4RQ2xYbLfynL
  Added 11,000 tracks to "Big List 1466376055_1"
Created playlist "Big List 1466376055_2", ID=6xO9P9CpcmAGhGLKImPLFs
  Added 304 tracks to "Big List 1466376055_2"
